# SQL Snippets (WIP)

## Fetching historical data for specific dates

Some tables in the warehouse--like those in `gtfs_schedule_type2`--capture the full
history of data, as it existed every day in the past. In order to do this, they
don't save copies of the full data every day, but log changes to the data using
a `calitp_extracted_at`, and `calitp_deleted_at` column.

In order to get the data for a given day, you need to filter to keep data where..

* `calitp_extracted_at` was earlier than or on the target date.
* `calitp_deleted_at` is later than the target date.

## A single date

In [1]:
from calitp.tables import tbl
from myst_nb import glue
from calitp import query_sql
from siuba import *
import pandas as pd
import calitp.magics

In [2]:
%%sql -m
SELECT *
FROM `gtfs_schedule_type2.feed_info`
WHERE
    calitp_extracted_at >= "2021-06-01"
    AND COALESCE(calitp_deleted_at, "2099-01-01") > "2021-06-01"

```SQL
SELECT *
FROM `gtfs_schedule_type2.feed_info`
WHERE
    calitp_extracted_at >= "2021-06-01"
    AND COALESCE(calitp_deleted_at, "2099-01-01") > "2021-06-01"

```

,calitp_itp_id,calitp_url_number,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email,feed_contact_url,calitp_extracted_at,calitp_deleted_at,calitp_hash
0,200,0,511 SF Bay,https://511.org/open-data/transit,en,None,20210829,20220101,20210828,None,None,2021-08-29,2021-09-13,UdQt0KWgXlYvMPYgJwF6HQ==
1,108,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210909,20220701,UTC: 09-Sep-2021 19:45,support+test+humboldtcounty-ca-us@trilliumtran...,http://support.trilliumtransit.com,2021-09-09,2021-09-13,AS9JK2VI09QWpc9MKCwL7w==
2,135,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210909,20220701,UTC: 09-Sep-2021 19:45,support+test+humboldtcounty-ca-us@trilliumtran...,http://support.trilliumtransit.com,2021-09-09,2021-09-13,6ALSwmDgitkbDzWbxQ0Qww==
3,42,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210909,20220701,UTC: 09-Sep-2021 19:45,support+test+humboldtcounty-ca-us@trilliumtran...,http://support.trilliumtransit.com,2021-09-09,2021-09-13,vlaYTVImTjaV3Nr0Aatkeg==
4,18,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210909,20220701,UTC: 09-Sep-2021 19:45,support+test+humboldtcounty-ca-us@trilliumtran...,http://support.trilliumtransit.com,2021-09-09,2021-09-13,ddtlovwoER2lcB5+ndaUIg==
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,218,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210820,20220601,UTC: 20-Aug-2021 20:26,support+test+vinetransit-ca-us@trilliumtransit...,http://support.trilliumtransit.com,2021-08-20,2021-11-03,cWL48eRINEOFPTuVA8Eb+A==
654,23,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20211007,20221001,UTC: 07-Oct-2021 21:23,support+test+auburntransit-ca-us@trilliumtrans...,http://support.trilliumtransit.com,2021-10-07,2021-11-03,671nnZsjUDsfXMlxpblxuQ==
655,278,0,MTS,http://www.sdmts.com,EN,None,None,None,v1 - Sept 5. 2021 svc change merged file,None,None,2021-08-23,2021-08-28,RdEZ6rQsk2F36zca38stOA==
656,200,0,511 SF Bay,https://511.org/open-data/transit,en,None,20210614,20210911,20210812,None,None,2021-08-13,2021-08-28,IwFAPEtcEwZhlexkQ4v2ng==


Note that `COALESCE` lets us fill in NULL deleted at values to be far in the future.
This is used because when deleted at is missing, it reflects the most recent data
(i.e. data that hasn't been deleted yet).
Because `NULL < "2021-06-01"` is `false`, we need to fill it in with a far-future date,
so it evaluates to `true`.

## Multiple dates

In order to do it for a range of dates, you can use a JOIN. This is shown below.

In [3]:
%%sql -m
SELECT *
FROM `gtfs_schedule_type2.feed_info` FI
JOIN `views.dim_date` D
    ON FI.calitp_extracted_at <= D.full_date
        AND COALESCE(FI.calitp_deleted_at, "2099-01-01") > D.full_date
WHERE
    D.full_date BETWEEN "2021-06-01" AND "2021-06-07"

```SQL
SELECT *
FROM `gtfs_schedule_type2.feed_info` FI
JOIN `views.dim_date` D
    ON FI.calitp_extracted_at <= D.full_date
        AND COALESCE(FI.calitp_deleted_at, "2099-01-01") > D.full_date
WHERE
    D.full_date BETWEEN "2021-06-01" AND "2021-06-07"

```

,calitp_itp_id,calitp_url_number,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email,...,month_name,week_day,day_name,day_is_weekday,is_quarter_start,is_month_start,is_in_past,is_in_past_or_present,is_in_future,is_gtfs_schedule_range
0,337,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201214,20210701,UTC: 15-Dec-2020 00:41,support+test+thousandoaks-ca-us@trilliumtransi...,...,June,0,Sunday,0,False,False,True,True,False,True
1,337,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201214,20210701,UTC: 15-Dec-2020 00:41,support+test+thousandoaks-ca-us@trilliumtransi...,...,June,1,Monday,1,False,False,True,True,False,True
2,337,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201214,20210701,UTC: 15-Dec-2020 00:41,support+test+thousandoaks-ca-us@trilliumtransi...,...,June,2,Tuesday,1,False,True,True,True,False,True
3,337,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201214,20210701,UTC: 15-Dec-2020 00:41,support+test+thousandoaks-ca-us@trilliumtransi...,...,June,4,Thursday,1,False,False,True,True,False,True
4,337,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201214,20210701,UTC: 15-Dec-2020 00:41,support+test+thousandoaks-ca-us@trilliumtransi...,...,June,6,Saturday,0,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,42,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210308,20210630,UTC: 09-Mar-2021 00:34,support+test+humboldtcounty-ca-us@trilliumtran...,...,June,2,Tuesday,1,False,True,True,True,False,True
1305,42,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210308,20210630,UTC: 09-Mar-2021 00:34,support+test+humboldtcounty-ca-us@trilliumtran...,...,June,4,Thursday,1,False,False,True,True,False,True
1306,42,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210308,20210630,UTC: 09-Mar-2021 00:34,support+test+humboldtcounty-ca-us@trilliumtran...,...,June,6,Saturday,0,False,False,True,True,False,True
1307,42,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210308,20210630,UTC: 09-Mar-2021 00:34,support+test+humboldtcounty-ca-us@trilliumtran...,...,June,3,Wednesday,1,False,False,True,True,False,True
